In [29]:
import pandas as pd
import dbconfig as dbConf

## 총합 양수 음수 갯수 구하는 함수

In [102]:
def pstv_ngtv_cnt(sum_num):
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    sql = (f"""
            select sum_num
                 , sum(case when diff > 0 then 1 end) as pstv
                 , sum(case when diff < 0 then 1 end) as ngtv
              from (
            select a.*
                 , ifnull((select (b.n1 + b.n2 + b.n3 + b.n4 + b.n5 + b.n6) - sum_num from temp b where b.seq = (a.seq + 1)),0) as diff
              from (
                    select a.seq
                         , a.n1, a.n2, a.n3, a.n4, a.n5, a.n6
                         , (a.n1 + a.n2 + a.n3 + a.n4 + a.n5 + a.n6) as sum_num
                      from temp a
                   ) a
             where 1=1
               and sum_num = {sum_num}
                   ) b 
             group by sum_num
           """
           )

    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    #print(sql)
    source = zeroDb.select(sql) 
    zeroDb.closedb()

    return pd.DataFrame(source)

## 패턴으로 결과값 뽑기

In [103]:
def Patten_GetNumber(max_val, f1, f2, f3, f4):
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    sql = (f"""
             select sum(case when row_num = 1 then num end) as n1
                  , sum(case when row_num = 2 then num end) as n2
                  , sum(case when row_num = 3 then num end) as n3
                  , sum(case when row_num = 4 then num end) as n4
                  , sum(case when row_num = 5 then num end) as n5
                  , sum(case when row_num = 6 then num end) as n6
                  , sum(num)                                as num_sum
                  , ''                                      as bin
                  , sum(case when row_num = 1 then seq end) as f1
                  , sum(case when row_num = 2 then seq end) as f2
                  , sum(case when row_num = 3 then seq end) as f3
                  , sum(case when row_num = 4 then seq end) as f4
                  , sum(case when row_num = 5 then seq end) as f5
                  , sum(case when row_num = 6 then seq end) as f6
               from (
             select num
                  , @rownum:=@rownum+1 as row_num
                  , seq
               from (
                    select seq, num   
                      from (
                            select 3 as seq, num
                              from in_list 
                             where seq = {max_val} 
                               and val = 3 
                             order by rand() 
                             limit {f1}
                           ) as t3
                    union all
                    select seq, num   
                      from (
                            select 5 as seq, num
                              from in_list 
                             where seq = {max_val}
                               and val = 5 
                             order by rand() 
                             limit {f2}
                           ) as t5 
                    union all
                    select seq, num   
                      from (
                            select 10 as seq, num
                              from in_list 
                             where seq = {max_val}
                               and val = 10
                             order by rand() 
                             limit {f3}
                           ) as t10
                    union all
                    select seq, num   
                    from (
                          select 30 as seq, num
                            from in_list 
                           where seq = {max_val}
                             and val = 30 
                           order by rand() 
                           limit {f4}
                         ) as t30
                     order by num 
                 ) as a
                 , (SELECT @rownum:=0) TMP
                 ) as b 
             ;
           """
           )

    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    #print(sql)
    source = zeroDb.select(sql) 
    zeroDb.closedb()

    return pd.DataFrame(source)

## sql 다이렉트로 날릴수 있는 함수

In [104]:
def sql_direct(sql_list):
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    sql = sql_list

    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    #print(sql)
    source = zeroDb.select(sql) 
    zeroDb.closedb()

    return pd.DataFrame(source)

## 패턴 얻기

In [306]:
def GetPatten():
    # MDM DB 연결
    zeroDb = dbConf.DbConfig("zero")
    zeroDb.opendb()
    #print("접속완료")

    sql = ("""
           select stan
                , 3cnt
                , 5cnt
                , 10cnt
                , 30cnt
             from in_list_2 a
            order by 1 desc
            limit 10
           """
           )

    # 수집 대상 DB정보 및 테이블, 쿼리 조회
    source = zeroDb.select(sql) 
    zeroDb.closedb()

    return pd.DataFrame(source)

In [307]:
# 패턴 뽑기
df = GetPatten()

stan    = list(range(0,1))
cnt3    = list(range(0,1))
cnt5    = list(range(0,1))
cnt10   = list(range(0,1))
cnt30   = list(range(0,1))

for df_stan, df_cnt3, df_cnt5, df_cnt10, df_cnt30 in zip(df['stan'], df['3cnt'], df['5cnt'], df['10cnt'], df['30cnt']):
    stan.append(int(df_stan))
    cnt3.append(int(df_cnt3))
    cnt5.append(int(df_cnt5))
    cnt10.append(int(df_cnt10))
    cnt30.append(int(df_cnt30))

In [308]:
df

,stan,3cnt,5cnt,10cnt,30cnt
0,1041,0,2,2,2
1,1040,2,0,2,2
2,1039,2,0,3,1
3,1038,1,0,5,0
4,1037,1,1,3,1
5,1036,4,1,1,0
6,1035,2,1,3,0
7,1034,2,0,3,1
8,1033,1,0,3,2
9,1032,2,0,2,2


## 전체 결과 

In [299]:
# 전체 내용
sql = """
select a.seq
     , a.n1
     , a.n2
     , a.n3
     , a.n4
     , a.n5
     , a.n6
     , a.sum_num as 합계
     , a.diff    as 차이    
     , b.pstv    as plus
     , b.ngtv    as minus
     , ''        as 빈칸
     , c.3cnt
     , c.5cnt
     , c.10cnt
     , c.30cnt
  from (
select a.seq
	 , a.n1, a.n2, a.n3, a.n4, a.n5, a.n6
	 , (a.n1 + a.n2 + a.n3 + a.n4 + a.n5 + a.n6) as sum_num
	 , ifnull((select (b.n1 + b.n2 + b.n3 + b.n4 + b.n5 + b.n6) - (a.n1 + a.n2 + a.n3 + a.n4 + a.n5 + a.n6) 
				 from temp b 
				where b.seq = (a.seq + 1)),0) as diff
  from temp a 
       ) a left outer join (select sum_num
								 , ifnull(sum(case when diff > 0 then 1 end),0) as pstv
								 , ifnull(sum(case when diff < 0 then 1 end),0) as ngtv
							  from (
							select (a.n1 + a.n2 + a.n3 + a.n4 + a.n5 + a.n6) as sum_num
								 , ifnull((select (b.n1 + b.n2 + b.n3 + b.n4 + b.n5 + b.n6) - (a.n1 + a.n2 + a.n3 + a.n4 + a.n5 + a.n6) 
											 from temp b 
											where b.seq = (a.seq + 1)),0) as diff
							  from temp a
								   ) a
							 group by sum_num
						   ) b
					    on a.sum_num = b.sum_num 
		   left outer join in_list_2 c
                        on a.seq = c.stan 
 order by 1 desc
 limit 10
       
"""

sql_all_data = sql_direct(sql)

In [300]:
sql_all_data

,seq,n1,n2,n3,n4,n5,n6,합계,차이,plus,minus,빈칸,3cnt,5cnt,10cnt,30cnt
0,1041,6,7,9,11,17,18,68,0,1,0,,None,None,None,None
1,1040,8,16,26,29,31,36,146,-78,5,3,,2,0,2,2
2,1039,2,3,6,19,36,39,105,41,8,2,,2,0,3,1
3,1038,7,16,24,27,37,44,155,-50,3,11,,1,0,5,0
4,1037,2,14,15,22,27,33,113,42,7,2,,1,1,3,1
5,1036,2,5,22,32,34,45,140,-27,3,11,,4,1,1,0
6,1035,9,14,34,35,41,42,175,-35,2,13,,2,1,3,0
7,1034,26,31,32,33,38,40,200,-25,0,1,,2,0,3,1
8,1033,3,11,15,20,35,44,128,72,14,8,,1,0,3,2
9,1032,1,6,12,19,36,42,116,12,10,5,,2,0,2,2


-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------

## 데이터 뽑기

In [305]:
# 최신 패턴으로 결과 뽑기
data = Patten_GetNumber(stan[1],0,0,0,0)

f1 = 3
f2 = 2
f3 = 1
f4 = 0

for i in range(0, 5):
    data = data.append(Patten_GetNumber(stan[1]+1, f1, f2, f3, f4), ignore_index = True)
    
data

,n1,n2,n3,n4,n5,n6,num_sum,bin,f1,f2,f3,f4,f5,f6
0,None,None,None,None,None,None,None,,None,None,None,None,None,None
1,5,6,27,37,38,45,158,,5,3,3,3,10,5
2,8,20,22,24,31,45,150,,3,10,5,3,3,5
3,3,5,15,36,42,44,145,,3,5,5,3,10,3
4,7,11,22,26,31,34,131,,3,10,5,3,3,5
5,6,19,32,33,36,38,164,,3,3,5,5,3,10


In [174]:
# 넘는 건만 담는다.
data2 = Patten_GetNumber(stan[1],0,0,0,0)

f1 = 3
f2 = 2
f3 = 1
f4 = 0

for i in range(0, 10):
    fill = Patten_GetNumber(stan[1]+1, f1, f2, f3, f4)
    
    if fill.loc[0]['num_sum'] > 128:
        data2 = data2.append(fill)
    
data2

,n1,n2,n3,n4,n5,n6,num_sum,bin,f1,f2,f3,f4,f5,f6
0,None,None,None,None,None,None,None,,None,None,None,None,None,None
0,7,13,17,32,33,40,142,,5,10,5,3,3,3
0,2,7,12,35,38,41,135,,5,5,3,3,3,10
0,1,5,7,33,41,44,131,,3,5,5,3,10,3
0,8,17,24,32,35,36,152,,10,5,5,3,3,3
0,2,20,24,40,42,45,173,,5,3,5,3,3,10
0,19,20,24,29,30,32,154,,3,3,5,5,10,3
0,2,3,24,26,39,42,136,,5,3,5,3,10,3
